# 따세권 EDA
+ 구별 거치대 분포 현황 (구별 거치대 설치 비율이 편향되지 않은지)
>result1: 영등포-마포구, 강남-서초구에 편향됨을 알 수 있음
+ 구별 거치대 위치 현황 ( 따릉이 정류소는 대중교통과 인접성을 고려한다는 입지요소 확인)
>result2: 대부분 도로를 따라 위치하며, 버스정류장과 지하철 역 인접권에 위치한다
+ 구별 거치대 설치 대수 대비 거치대 이용건수 시각화 (수요와 공급이 불균형 한 지역구가 있는지 확인)
>result3 :
거치대가 많고 이용건수도 많은 지역 : 마포구, 영등포구(따세권) /
거치대가 많고 이용건수가 적은 지역 : 서초구, 강남구(따세권) /
거치대가 적고 이용건수가 많은 지역 : 중구,동작구,용산구(비따세권) /
거치대가 적고 이용건수도 적은 지역 : 강북구,도봉구,금천구(비따세권) 

+ 따세권으로 자리잡은 지역과 비따세권 지역 대여횟수와 반납횟수 비율(이용 형태 확인)
>result4 : 
거치대가 적고 이용건수도 적은 도봉구,강북구는 반납건수>대여건수 로 반납의 용도로 많이 사용한는 것을 알 수 있음 / 영등포구와 마포구는 반납과 대여가 비슷하게 발생함 / 반면 강남구와 서초구는 반납건수<대여건수로 대여의 용도로 많이 사용하는 것을 알 수 있음

+ 주간인구지수와 따릉이 이용형태의 연관성(거주인구와 유동인구) 
>result5 : 거치대가 적고 이용건수도 적은 도봉구,강북구는 반납건수>대여건수 로 반납의 용도로 많이 사용한는 것을 알 수 있음(상주인구가 많은 지역은 따릉이 거치대를 반납의 이용으로 많이 이용한다)

+ 직장인 상권과 관련이 있을까?
>result6: 따릉이 이용이 유동인구(통근)와 연관성 있는 것은 맞지만 아직 직장인 상권과는 크게 연관성 있어 보이지 않는다.

In [24]:
import pandas as pd
import numpy as np
import re
import random

In [25]:
info = pd.read_excel('data/bikestop_info.xlsx', encoding='utf8') # 데이터 파일 경로 수정

In [48]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 7 columns):
구        1163 non-null object
대여소번호    1163 non-null object
대여소명     1163 non-null object
주소       1163 non-null object
거치대      1163 non-null int64
위도       1163 non-null float64
경도       1163 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 63.7+ KB


In [27]:
info['대여소번호'] = info['대여소번호'].astype(object)

In [28]:
info.head()

,구,대여소번호,대여소명,주소,거치대,위도,경도
0,마포구,101,(구)합정동 주민센터,서울특별시 마포구 동교로8길 58,5,37.549561,126.905754
1,마포구,102,망원역 1번출구 앞,서울특별시 마포구 월드컵로 72,20,37.556000,126.910454
2,마포구,103,망원역 2번출구 앞,서울특별시 마포구 월드컵로 79,14,37.554951,126.910835
3,마포구,104,합정역 1번출구 앞,서울특별시 마포구 양화로 59,13,37.550629,126.914986
4,마포구,105,합정역 5번출구 앞,서울특별시 마포구 양화로 48,5,37.550007,126.914825


### 구별 거치대 총 설치 개수

In [29]:
temp_info=info.copy()

In [30]:
temp_info_1=temp_info.groupby(['구'])['거치대'].sum()
sum_temp_info=pd.DataFrame(temp_info_1)
sum_temp_info

,거치대
구,
강남구,845
강동구,595
강북구,361
강서구,528
관악구,526
광진구,576
구로구,581
금천구,480
노원구,615


In [31]:
import json
import folium

import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

geo_path = 'data/skorea_municipalities_geo_simple.json' # 데이터 파일 경로 수정 & 데이터 파일 추가
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [32]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

map.choropleth(
    geo_data = geo_str,
    data = sum_temp_info['거치대'],
    columns = ['구','거치대'],
               fill_color = 'YlGnBu',
               key_on = 'feature.id')
map

## >result1. 지역구별 거치대 설치 개수 편중된 상태임을 알 수 있음

In [33]:
map = folium.Map(location=[info['위도'].mean(), info['경도'].mean()], zoom_start=12, tiles='OpenStreetMap')

for n in info.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = info['구'][n] + ' - ' + info['대여소명'][n]
    
    # folium.Circle()로 변경 시에는 radius 값이 실제 m(meter) 단위를 의미하게 됩니다. (CircleMarker의 경우는 pixel 단위)
    folium.CircleMarker(
        location=[info['위도'][n], info['경도'][n]],
        radius=5,
        popup=popup_name,    # marker 위에 마우스 올릴 때 뜨는 popup
        fill=True,
    ).add_to(map)

map

## result2.정류소 위치를 살펴보면 대부분이 도로와 인근하고 있다
+ 대중교통과의 연계성(정류소 인근에 정류소와 지하철역이 있음) | 대중교통 데이터와 엮어보면 좋을 것
+ 대부분이 도로를 따라 설치되어 있다 | 정류장과 지하철역이 인접하지 않은 곳에서의 거치대 주변 환경적 요소 분석해보면 좋을 것

### 그렇다면 실제 거치대가 많이 설치된 곳에서 이용건수가 실제로 많을 것인가?
#### : 따릉이 거치장소의 효용성

In [34]:
# utf-8, euc-kr, cp949

use = pd.read_csv('data/bikeuse_18.csv',encoding='cp949') # 데이터 파일 경로 수정
use.head()

,대여일자,대여소번호,대여소,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이동시간(분)
0,'201801','108',' 서교동 사거리','정기','F','~10대',1,179.94,1.32,5680,36
1,'201801','1031',' 암사동 CBIS','정기','F','~10대',4,194.39,1.93,8320,49
2,'201801','1032',' 선사고등학교','정기','F','~10대',1,13.55,0.13,580,3
3,'201801','1033',' 고덕동 아남아파트','정기','F','~10대',2,58.63,0.55,2390,15
4,'201801','1253',' 오금역 3번 출구 뒤','정기','F','~10대',3,83.24,0.79,3390,26


In [35]:
use['대여소'] = use['대여소'].astype(str)

In [36]:
use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143982 entries, 0 to 143981
Data columns (total 11 columns):
대여일자       143982 non-null object
대여소번호      143977 non-null object
대여소        143982 non-null object
대여구분코드     143888 non-null object
성별         143982 non-null object
연령대코드      129832 non-null object
이용건수       143982 non-null int64
운동량        143982 non-null float64
탄소량        143982 non-null float64
이동거리(M)    143982 non-null int64
이동시간(분)    143982 non-null int64
dtypes: float64(2), int64(3), object(6)
memory usage: 12.1+ MB


In [37]:
use.iloc[:,:6] = use.iloc[:,:6].apply(lambda x: x.str.replace('\'', ''))

In [38]:
use.head()

,대여일자,대여소번호,대여소,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이동시간(분)
0,201801,108,서교동 사거리,정기,F,~10대,1,179.94,1.32,5680,36
1,201801,1031,암사동 CBIS,정기,F,~10대,4,194.39,1.93,8320,49
2,201801,1032,선사고등학교,정기,F,~10대,1,13.55,0.13,580,3
3,201801,1033,고덕동 아남아파트,정기,F,~10대,2,58.63,0.55,2390,15
4,201801,1253,오금역 3번 출구 뒤,정기,F,~10대,3,83.24,0.79,3390,26


In [39]:
info['대여소번호'] = info['대여소번호'].astype(str)

In [40]:
result = pd.merge(use, info, on=['대여소번호'], how='left') # SQL left outer join
result.head()

,대여일자,대여소번호,대여소,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이동시간(분),구,대여소명,주소,거치대,위도,경도
0,201801,108,서교동 사거리,정기,F,~10대,1,179.94,1.32,5680,36,마포구,서교동 사거리,서울특별시 마포구 양화로 93,10.0,37.552746,126.918617
1,201801,1031,암사동 CBIS,정기,F,~10대,4,194.39,1.93,8320,49,강동구,암사동 CBIS,서울특별시 암사동 440-13,15.0,37.555851,127.129898
2,201801,1032,선사고등학교,정기,F,~10대,1,13.55,0.13,580,3,강동구,선사고등학교,서울특별시 암사동 445,20.0,37.556728,127.136208
3,201801,1033,고덕동 아남아파트,정기,F,~10대,2,58.63,0.55,2390,15,강동구,고덕동 아남아파트,서울특별시 고덕동 673,10.0,37.557991,127.144707
4,201801,1253,오금역 3번 출구 뒤,정기,F,~10대,3,83.24,0.79,3390,26,송파구,오금역 3번 출구 뒤,서울특별시 송파구 가락동 10-15,10.0,37.501652,127.128181


In [41]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143982 entries, 0 to 143981
Data columns (total 17 columns):
대여일자       143982 non-null object
대여소번호      143977 non-null object
대여소        143982 non-null object
대여구분코드     143888 non-null object
성별         143982 non-null object
연령대코드      129832 non-null object
이용건수       143982 non-null int64
운동량        143982 non-null float64
탄소량        143982 non-null float64
이동거리(M)    143982 non-null int64
이동시간(분)    143982 non-null int64
구          136698 non-null object
대여소명       136698 non-null object
주소         136698 non-null object
거치대        136698 non-null float64
위도         136698 non-null float64
경도         136698 non-null float64
dtypes: float64(5), int64(3), object(9)
memory usage: 19.8+ MB


In [42]:
result1 = result.copy()

In [43]:
result1=result.groupby(['구'])['이용건수'].sum()
temp_result1=pd.DataFrame(result1)
temp_result1.reset_index(inplace=True)

In [44]:
temp_result1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
구       25 non-null object
이용건수    25 non-null int64
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


In [45]:
num_use=pd.merge(sum_temp_info,temp_result1, on='구')
num_use

,구,거치대,이용건수
0,강남구,845,133282
1,강동구,595,95725
2,강북구,361,53798
3,강서구,528,109777
4,관악구,526,114995
5,광진구,576,220031
6,구로구,581,110421
7,금천구,480,44181
8,노원구,615,121560
9,도봉구,250,34972


In [46]:
# pip install plotnine==0.5.1

from plotnine import *

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc # rc == run configure(configuration file)
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

ModuleNotFoundError: No module named 'plotnine'

In [ ]:
(ggplot(num_use)
 + aes(x='구', y='이용건수', fill='거치대')
 + geom_col()
 + theme(text=element_text(family=font_name), # 'NanumBarunGothic' -> font_name
        axis_text_x=element_text(rotation=60))
)

### result3 거치대가 많다고 이용률이 다 높은건 아니다. 수요와 공급이 편향된 몇몇 곳이 보인다
+ 거치대가 많고 이용건수도 많은 지역 : 마포구, 영등포구(따세권)
+ 거치대가 많고 이용건수가 적은 지역 : 서초구, 강남구(따세권)
*(서초구와 강남구는 심지어 5대 거점도 아니었다)*
+ 거치대가 적고 이용건수가 많은 지역 : 중구,동작구,용산구(비따세권)
+ 거치대가 적고 이용건수도 적은 지역 : 강북구,도봉구,금천구(비따세권)

#### 서초구와 강남구는 이용건수 대비 거치대가 많은 상황(수요<공급)
#### 이용건수와 거치대 모두 많은 지역은 마포구와 영등포구(수요~공급)
#### 두드러지진 않지만 종로구,광진구 혹은 중구,동작구,용산구 수요>공급
#### 강북구,도봉구,금천구는 이용건수와 거치대 설치 모두 낮은 상황


# 따세권으로 자리잡은 지역과 비따세권 지역 대여횟수와 반납횟수 비율(이용 형태)
+ 대여보다 반납이 많은 지역은 유동인구보다 거주인구가 많을 것이다.
+ 대여보다 반납이 적은 지역은 유동인구보다 거주인구가 적을 것이다.

In [ ]:
bike_stat= pd.read_excel('data/bike_stat18.xlsx', encoding='utf-8') # 데이터 파일 경로 수정

In [ ]:
bike_stat.head(3)

In [ ]:
bike_stat.iloc[:,:3] = bike_stat.iloc[:,:3].apply(lambda x: x.str.replace('\'', ''))
bike_stat.head()

In [ ]:
bike_stat.info()

In [ ]:
tmp = bike_stat.copy()

In [ ]:
tmp1=tmp.groupby(['대여소번호','대여소'])['대여건수','반납건수'].sum()
temp_result1=pd.DataFrame(tmp1)
temp_result1.reset_index(inplace=True)
temp_result1

In [ ]:
merge = pd.merge(temp_result1, info, on=['대여소번호'], how = 'left')
merge.head()

In [ ]:
merge['이용형태'] = (merge['반납건수'] / merge['대여건수']).round(3)

In [ ]:
merge.head()

#### 대여건수 대비 반납건수인 이용형태의 값이 클수록 거치대에서 '반납'이 자주 일어난다는 뜻이다.

In [ ]:
merge.isnull().sum()

In [ ]:
mp_ydp_use = merge[(merge['구'] == '마포구') | (merge['구'] == '영등포구')]
mp_ydp_use.describe()

In [ ]:
sc_gn_use = merge[(merge['구'] == '서초구') | (merge['구'] == '강남구')]
sc_gn_use.describe()

In [ ]:
jg_dj_use = merge[(merge['구'] == '중구') | (merge['구'] == '동작구')]
jg_dj_use.describe()    

In [ ]:
mp_ydp_use = merge[(merge['구'] == '도봉구') | (merge['구'] == '강북구')]
mp_ydp_use.describe()

#### 거치대 수도 작고, 이용건수도 적은 도봉구 강북구의 경우 '반납/대여 > 1' 
#### 도봉구와 강북구의 거치대는 대여보다 반납의 용도로 이용하는 경우가 많은 것을 알 수 있음

In [ ]:
merge.groupby('구').mean()

### 구별 평균 이용형태 평균값

In [ ]:
import numpy as np
usetype = merge.pivot_table(index='구', values='이용형태',  aggfunc=np.mean).round(2)
usetype.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 한글 깨짐을 방지하기 위해 아래와 같이 코드를 추가합니다.
from matplotlib import font_manager, rc

# 현재 로컬 컴퓨터 내의 폰트 파일 경로를 지정해줍니다.
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name() 
rc('font', family=font_name)

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(usetype.sort_values(by='이용형태', ascending=False), annot=True, fmt='f', linewidths=.5)
plt.title('반납건수/대여건수 (거치대 이용형태) ')
plt.show()

In [ ]:
usetype.describe()

In [ ]:
usetype[usetype['이용형태'] > 1]

#### 이용형태>1인 지역은 반납건수>이용건수 인 지역으로 따릉이를 최초 이용한다기보다 다른데서 타고 반납하는 경우가 많을 것이다. 
#### (예를 들어 퇴근하는 직장인이 역에서 내려 따릉이를 타고 정차한다거나...)

## 그렇다면 상주인구와 주간인구(유동인구)를 알아보자

In [ ]:
popmove = pd.read_excel('data/popstat.xlsx', encoding='utf-8') # 파일 경로 수정
popmove.head()

In [ ]:
pop_stat_usetype=pd.merge(popmove,usetype, on=['구'], how='left')
pop_stat_usetype.head()

In [ ]:
import plotnine
plotnine.options.figure_size = (15, 7)

(ggplot(popmove)
 + aes(x='구', y='주간인구지수')
 + geom_point()
 + ggtitle('주간인구지수')
 + theme(text=element_text(family=font_name),
         axis_text_x=element_text(rotation=60))
)

In [ ]:
usetype1 = usetype.copy()
usetype1['이용형태'] = usetype1['이용형태'].astype(int)
usetype1

#### 반납건수가 많은 지역과 이외지역을 0과 1로 구분

In [ ]:
fpop_stat_usetype=pd.merge(popmove,usetype1, on=['구'], how='left')
fpop_stat_usetype.head()

In [ ]:
(ggplot(fpop_stat_usetype)
 + aes(x='구', y='주간인구지수', color='이용형태')
 + geom_point()
 + stat_smooth()
 +facet_wrap('~이용형태')
 + ggtitle('주간인구지수와 이용형태 관계')
 + theme(text=element_text(family=font_name),
         axis_text_x=element_text(rotation=70))
 
)

#### 거치대가 적고 이용건수도 적은 도봉구,강북구는 반납건수>대여건수 로 반납의 용도로 많이 사용한는 것을 알 수 있음

# 직장인 상권과 관련이 있을까?

In [ ]:
bizpop = pd.read_excel('data/bizpop18.xlsx', encoding='utf-8') # 파일 경로 수정
bizpop.head(7)

### 도로명 주소로부터 위도,경도 위치데이터 가져오기(googlemaps 이용)

In [ ]:
# !pip install googlemaps
!pip show googlemaps==2.4.6

In [ ]:
# 악용을 방지하기 위해 개인 키를 지웠습니다.

import googlemaps
gmaps_key = '~~~' # type your googlemaps API key from https://developers.google.com/maps/documentation/geocoding/get-api-key (https://goo.gl/mU5NYK)
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
name = '휘경로2길'
gmaps.geocode(name, language='ko')

In [ ]:
# bizpop_name의 지역명이 너무 많아(+1,600) geocode() 함수가 timeout 을 일으키는 경우가 있으므로 아래와 같이 쪼개어 진행합니다.

bizpop_name = bizpop['상권 코드 명'] 
bizpop_address = []
bizpop_lat = []
bizpop_lng = []

bizpop_name_1 = bizpop_name[:400]
bizpop_name_2 = bizpop_name[400:800]
bizpop_name_3 = bizpop_name[800:1200]
bizpop_name_4 = bizpop_name[1200:]

In [ ]:
def geocode_bizpop(bizpop_temp, list_address, list_lat, list_lng):
    for name in [n for n in bizpop_temp if type(n) == str]:
        tmp = gmaps.geocode(name, language='ko')
        if tmp != []:
            result = (tmp[0].get("formatted_address"))    
            list_address.append(result)

            tmp_loc = tmp[0].get("geometry")
            list_lat.append(tmp_loc['location']['lat'])
            list_lng.append(tmp_loc['location']['lng'])

In [ ]:
geocode_bizpop(bizpop_name_1, bizpop_address, bizpop_lat, bizpop_lng)

In [ ]:
geocode_bizpop(bizpop_name_2, bizpop_address, bizpop_lat, bizpop_lng)

In [ ]:
geocode_bizpop(bizpop_name_3, bizpop_address, bizpop_lat, bizpop_lng)

In [ ]:
geocode_bizpop(bizpop_name_4, bizpop_address, bizpop_lat, bizpop_lng)

In [ ]:
bizpop_dict = {'상권주소': bizpop_address, '위도': bizpop_lat, '경도': bizpop_lng}
bizpop_full = pd.DataFrame(bizpop_dict)
bizpop_full.tail()

In [ ]:
bizpop_full['상권 코드 명'] = bizpop_full['상권주소'].apply(lambda x : x.split(' ')[-1])
bizpop_full.head()

In [ ]:
fbizpop=pd.merge(bizpop_full,bizpop, on=['상권 코드 명'], how='left')
fbizpop.head(3)

In [ ]:
map = folium.Map(location=[fbizpop['위도'].mean(), fbizpop['경도'].mean()], zoom_start=12, tiles='OpenStreetMap')

for n in fbizpop.index:
           
    folium.features.CircleMarker(
        location=[fbizpop['위도'][n], fbizpop['경도'][n]],
        radius=1,
        fill=False,
    ).add_to(map)

map

In [ ]:
bizpop_final=fbizpop.dropna() 

In [ ]:
bizpop_final.info()

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=10)

map.choropleth(
    geo_data = geo_str,
    data = sum_temp_info['거치대'],
    columns = ['구','거치대'],
               fill_color = 'PuRd',
               key_on = 'feature.id')

for n in fbizpop.index:
           
    folium.features.CircleMarker(
        location=[fbizpop['위도'][n], fbizpop['경도'][n]],
        radius=1,
        fill=False,
    ).add_to(map)

map

#### 직장인 상권과는 연관성이 두드러지지 않음

## 추가적인 분석을 계속 해본다면....

+ 상권 소비 유형과의 연관성(식음료,교육,여가-문화생활 등)
+ 공원 및 공공문화시설 등의 분포와도 함께 분석해보면 좋을 것
> 따릉이 대여소 후보지 조사 양식에 포함된 지하철역, 버스정류소,아파트,주거지역,상가,관공서,공원,대학가 등의 요소들을 분석해 대여소 주변 토지 이용 형태와 따릉이 이용률 간 연관성을 분석해 볼 수 있음(어떤 요소가 따릉이 이용률에 가장 큰 영향을 미치는지)
+ 지역의 통근 유입, 통근 유출 인구와 따릉이 대여 및 반납 현황 분석으로 대중교통과의 연계성이 유효한지 분석